In [2]:
import requests

url = "https://leetcode.com/graphql/"
headers = {
  'authority': 'leetcode.com',
  'accept': '*/*',
  'accept-language': 'en-US,en;q=0.9',
  'authorization': '',
  'content-type': 'application/json',
  'cookie': '',
  'dnt': '1',
  'origin': 'https://leetcode.com',
  'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="101", "Microsoft Edge";v="101"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32',
  'x-csrftoken': '',
}

def getListOfProblems():
    data = {"query":"\n    query problemsetQuestionList($categorySlug: String, $limit: Int, $skip: Int, $filters: QuestionListFilterInput) {\n  problemsetQuestionList: questionList(\n    categorySlug: $categorySlug\n    limit: $limit\n    skip: $skip\n    filters: $filters\n  ) {\n    total: totalNum\n    questions: data {\n      acRate\n      difficulty\n      freqBar\n      frontendQuestionId: questionFrontendId\n      isFavor\n      paidOnly: isPaidOnly\n      status\n      title\n      titleSlug\n      topicTags {\n        name\n        id\n        slug\n      }\n      hasSolution\n      hasVideoSolution\n    }\n  }\n}\n    ","variables":{"categorySlug":"","skip":0,"limit":3000,"filters":{}}}
    response = requests.post(url, headers=headers, json=data)
    return response.json()["data"]["problemsetQuestionList"]["questions"]

problemsList = getListOfProblems()


In [3]:
len(problemsList)

3000

In [4]:
import re

def getPlaygroundData(uuid):
    requestData = {"operationName":"fetchPlayground","variables":{},"query":"query fetchPlayground {\n  playground(uuid: \"" + uuid + "\") {\n    testcaseInput\n    name\n    isUserOwner\n    isLive\n    showRunCode\n    showOpenInPlayground\n    selectedLangSlug\n    isShared\n    __typename\n  }\n  allPlaygroundCodes(uuid: \"cvvgJGBX\") {\n    code\n    langSlug\n    __typename\n  }\n}\n"}
    response = requests.post(url, headers=headers, json=requestData)
    return response.json()["data"]


def getProblemStatement(idx, titleSlug):
    try:
        questionRequestData = {"operationName":"questionData","variables":{"titleSlug": titleSlug},"query":"query questionData($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    questionFrontendId\n    boundTopicId\n    title\n    titleSlug\n    content\n    translatedTitle\n    translatedContent\n    isPaidOnly\n    difficulty\n    likes\n    dislikes\n    isLiked\n    similarQuestions\n    exampleTestcases\n    categoryTitle\n    contributors {\n      username\n      profileUrl\n      avatarUrl\n      __typename\n    }\n    topicTags {\n      name\n      slug\n      translatedName\n      __typename\n    }\n    companyTagStats\n    codeSnippets {\n      lang\n      langSlug\n      code\n      __typename\n    }\n    stats\n    hints\n    solution {\n      id\n      canSeeDetail\n      paidOnly\n      hasVideoSolution\n      paidOnlyVideo\n      __typename\n    }\n    status\n    sampleTestCase\n    metaData\n    judgerAvailable\n    judgeType\n    mysqlSchemas\n    enableRunCode\n    enableTestMode\n    enableDebugger\n    envInfo\n    libraryUrl\n    adminUrl\n    challengeQuestion {\n      id\n      date\n      incompleteChallengeCount\n      streakCount\n      type\n      __typename\n    }\n    __typename\n  }\n}\n"}
        questionResponse = requests.post(url, headers=headers, json=questionRequestData).json()
        question = questionResponse["data"]["question"]
        id = question["questionId"]

        if (questionResponse["data"]["question"]["solution"] != None):
            solutionRequestData = {"operationName":"QuestionNote","variables":{"titleSlug": titleSlug},"query":"query QuestionNote($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    article\n    solution {\n      id\n      content\n      contentTypeId\n      canSeeDetail\n      paidOnly\n      hasVideoSolution\n      paidOnlyVideo\n      rating {\n        id\n        count\n        average\n        userRating {\n          score\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"}
            solutionResponse = requests.post(url, headers=headers, json=solutionRequestData).json()

            playgroundUrls = re.findall(r'https://leetcode.com/playground/(.+)/shared', solutionResponse["data"]["question"]["solution"]["content"])
            playgroundData = {}

            for codeUrl in playgroundUrls:
                codeUrl = codeUrl.replace("https://leetcode.com/playground/", "")
                codeUrl = codeUrl.replace("/shared", "")
                playgroundData[codeUrl] = getPlaygroundData(codeUrl)
            
            solution = solutionResponse["data"]["question"]["solution"]
        else:
            solution = None
            solutionResponse = None
            playgroundData = None
       
        problemData = {
            "id": id,
            "question": question,
            "solution": solution,
            "questionCode": titleSlug,
            "playgroundData": playgroundData
        }
        
        return True, problemData
    except Exception as err:
        return False, idx

In [5]:
import json
import time

lastError = [2335]

while lastError:
    startIdx = lastError.pop()
    print("Starting again with = ", startIdx)

    for idx, question in enumerate(problemsList[startIdx:]):
        success, res = getProblemStatement(startIdx + idx, question["titleSlug"])

        if success:
            json_object = json.dumps(res, indent = 4)
            file_name = f"{res['id'].zfill(4)}_{question['titleSlug']}.json"
            
            with open('problems/' + file_name, "w") as outfile:
                outfile.write(json_object)
        else:
            if res == startIdx:
                res += 1
            lastError.append(res)
            print("Waiting...")
            time.sleep(20)
            print("Wait stopped...")
            break
    

Starting again with =  2335


Waiting...
Wait stopped...
Starting again with =  2336
Waiting...
Wait stopped...
Starting again with =  2347
Waiting...
Wait stopped...
Starting again with =  2348
Waiting...
Wait stopped...
Starting again with =  2351
Waiting...
Wait stopped...
Starting again with =  2352
Waiting...
Wait stopped...
Starting again with =  2354
Waiting...
Wait stopped...
Starting again with =  2355
Waiting...
Wait stopped...
Starting again with =  2356
Waiting...
Wait stopped...
Starting again with =  2359
Waiting...


KeyboardInterrupt: 